In [5]:
# from sklearn.ensemble import StackingClassifier
# from sklearn.linear_model import LogisticRegression
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.svm import SVC
# from xgboost import XGBClassifier
# from sklearn.neural_network import MLPClassifier
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.preprocessing import StandardScaler
# from sklearn.metrics import accuracy_score, classification_report
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import LabelEncoder
# import pandas as pd
# from sklearn.model_selection import GridSearchCV

# # Load your dataset
# df = pd.read_csv("data\data.csv") 


# label_encoder = LabelEncoder()
# df['label'] = label_encoder.fit_transform(df['label'])

# X = df.drop(['label', 'filename'], axis=1)
# y = df['label']

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)




# # Define your base classifiers
# base_classifiers = [
#     ('rf', RandomForestClassifier(random_state=42)),
#     ('svc', SVC(probability=True, random_state=42)),
#     ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')),
#     ('mlp', MLPClassifier(random_state=42)),
#     ('knn', KNeighborsClassifier()),
#     ('log_clf', LogisticRegression(random_state=42))
# ]

# # Meta-classifier
# meta_classifier = LogisticRegression(random_state=42)

# # Initialize the Stacking Classifier
# stacking_clf = StackingClassifier(estimators=base_classifiers, final_estimator=meta_classifier, cv=5)

# # Parameter grid for GridSearchCV
# param_grid = {
#     'rf__n_estimators': [100, 200, 300],
#     'rf__max_depth': [10, None, 5],
#     'svc__C': [0.1, 1, 10],
#     'xgb__n_estimators': [50, 100, 150],
#     'xgb__learning_rate': [0.01, 0.1, 0.2],
#     'mlp__hidden_layer_sizes': [(50,), (100,), (50, 50)],
#     'mlp__max_iter': [500, 1000, 1500],
#     'knn__n_neighbors': [3, 5, 7],
#     'log_clf__C': [0.1, 1, 10],
#     'final_estimator__C': [0.1, 1, 10]
# }

# # Setup GridSearchCV
# grid_search = GridSearchCV(estimator=stacking_clf, param_grid=param_grid, cv=3, scoring='accuracy', verbose=2, n_jobs=-1)

# # Perform the search
# grid_search.fit(X_train_scaled, y_train)

# # Best parameters and score
# print("Best parameters found: ", grid_search.best_params_)
# print("Best cross-validation score: {:.2f}".format(grid_search.best_score_))

# # Evaluate on the test set
# y_pred = grid_search.predict(X_test_scaled)
# print("Accuracy on test set: {:.2f}".format(accuracy_score(y_test, y_pred)))
# print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))


In [6]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from sklearn.model_selection import GridSearchCV

# Load your dataset
df = pd.read_csv("data\data.csv") 


label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])

X = df.drop(['label', 'filename'], axis=1)
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [7]:
# from sklearn.ensemble import StackingClassifier
# from sklearn.model_selection import RandomizedSearchCV
# from scipy.stats import randint as sp_randint
# from scipy.stats import uniform

# # Assuming the imports and data preparation steps are done as before

# # Base classifiers with example classifiers
# base_classifiers = [
#     ('rf', RandomForestClassifier(random_state=42)),
#     ('svc', SVC(probability=True, random_state=42)),
#     ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')),
#     ('mlp', MLPClassifier(random_state=42)),
#     ('knn', KNeighborsClassifier()),
#     ('log_clf', LogisticRegression(random_state=42))
# ]


# # Meta-classifier
# meta_classifier = LogisticRegression(random_state=42)

# # Initialize the Stacking Classifier
# stacking_clf = StackingClassifier(estimators=base_classifiers, final_estimator=meta_classifier, cv=5)

# # Parameter distributions
# param_dist = {
#     'rf__n_estimators': sp_randint(100, 500),  # Number of trees in random forest
#     'rf__max_depth': [None, 10, 20, 30],  # Maximum depth of trees
#     'svc__C': uniform(0.1, 10),  # Penalty parameter C of the SVC
#     'xgb__n_estimators': sp_randint(50, 150),  # Number of gradient boosted trees
#     'xgb__learning_rate': uniform(0.01, 0.2),  # Boosting learning rate
#     'mlp__hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50)],  # MLP architecture
#     'mlp__alpha': uniform(0.0001, 0.001),  # L2 penalty (regularization term) parameter
#     'knn__n_neighbors': sp_randint(3, 10),  # Number of neighbors to use
#     'log_clf__C': uniform(0.1, 10),  # Inverse of regularization strength in LogisticRegression
#     'final_estimator__C': uniform(0.1, 10),  # Regularization in the final meta-classifier
# }


# # Initialize RandomizedSearchCV with the Stacking Classifier and parameter distribution
# random_search = RandomizedSearchCV(stacking_clf, param_distributions=param_dist, n_iter=100, cv=5, verbose=2, random_state=42, n_jobs=-1)

# # Perform the search
# random_search.fit(X_train_scaled, y_train)

# # Best parameters and score
# print("Best parameters found: ", random_search.best_params_)
# print("Best cross-validation score: {:.2f}".format(random_search.best_score_))

# # Evaluate on the test set
# y_pred = random_search.predict(X_test_scaled)
# print("Accuracy on test set: {:.2f}".format(accuracy_score(y_test, y_pred)))
# print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))


In [8]:
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

# Reinitialize the classifiers with the best hyperparameters
rf_best = RandomForestClassifier(n_estimators=317, max_depth=10, random_state=42)
svc_best = SVC(C=7.003948286293653, probability=True, random_state=42)
xgb_best = XGBClassifier(n_estimators=96, learning_rate=0.01786242796821979, use_label_encoder=False, eval_metric='mlogloss')
mlp_best = MLPClassifier(alpha=0.0009942172628959257, hidden_layer_sizes=(50,), max_iter=1000, random_state=42)
knn_best = KNeighborsClassifier(n_neighbors=3)
log_clf_best = LogisticRegression(C=3.659914842719861, max_iter=5000, random_state=42)

# Meta-classifier with the best hyperparameters
meta_classifier_best = LogisticRegression(C=1.3063587110060082, random_state=42)

# Create the Stacking Classifier with the best hyperparameters
stacking_clf_best = StackingClassifier(
    estimators=[
        ('rf', rf_best),
        ('svc', svc_best),
        ('xgb', xgb_best),
        ('mlp', mlp_best),
        ('knn', knn_best),
        ('log_clf', log_clf_best)
    ],
    final_estimator=meta_classifier_best,
    cv=5
)

# Assuming X_train_scaled, y_train, X_test_scaled, y_test are already defined and prepared

# Fit the Stacking Classifier with the best hyperparameters
stacking_clf_best.fit(X_train_scaled, y_train)

# Predict and evaluate the model
y_pred_best = stacking_clf_best.predict(X_test_scaled)
print(f"Accuracy: {accuracy_score(y_test, y_pred_best)}")
print(classification_report(y_test, y_pred_best, target_names=label_encoder.classes_))


C:\Users\bryso\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\bryso\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\bryso\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\brys

Accuracy: 0.665
              precision    recall  f1-score   support

       blues       0.75      0.75      0.75        20
   classical       0.87      1.00      0.93        13
     country       0.59      0.74      0.66        27
       disco       0.60      0.43      0.50        21
      hiphop       0.52      0.73      0.61        15
        jazz       0.75      0.68      0.71        22
       metal       0.80      0.80      0.80        25
         pop       0.86      0.92      0.89        13
      reggae       0.63      0.52      0.57        23
        rock       0.35      0.29      0.32        21

    accuracy                           0.67       200
   macro avg       0.67      0.69      0.67       200
weighted avg       0.66      0.67      0.66       200



In [1]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from sklearn.decomposition import PCA
import jsonpickle

# Correct the path to the new dataset
path = 'data/features_3_sec.csv'  # Adjusted to use forward slashes

# Load the new dataset
df = pd.read_csv(path)

# Encode the labels
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])

# Define features and labels
X = df.drop(["label", "filename", 'length'], axis=1)
y = df['label']


# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

scaler_json = jsonpickle.encode(scaler)

# Save the JSON string to a file
with open("scaler.json", "w") as file:
    file.write(scaler_json)



In [2]:
# Base classifiers with example classifiers
base_classifiers = [
    ("rf", RandomForestClassifier(random_state=42)),
    ("svc", SVC(probability=True, random_state=42)),
    ("xgb", XGBClassifier(eval_metric="mlogloss", random_state=42)),
    (
        "mlp",
        MLPClassifier(
            alpha=0.0009942172628959257,
            hidden_layer_sizes=(50,),
            max_iter=2000,
            random_state=42,
        ),
    ),
    ("knn", KNeighborsClassifier()),
    (
        "log_clf",
        LogisticRegression(
            C=3.659914842719861, max_iter=5000, solver="saga", random_state=42
        ),
    ),
]


# Meta-classifier
meta_classifier = LogisticRegression(random_state=42)

# Initialize the Stacking Classifier
stacking_clf = StackingClassifier(
    estimators=base_classifiers, final_estimator=meta_classifier, cv=5
)

# Fit the Stacking Classifier on the scaled training data
stacking_clf.fit(X_train_scaled, y_train)

# Predict on the scaled test set and evaluate
y_pred = stacking_clf.predict(X_test_scaled)
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))


from joblib import dump, load

# Save the model to disk
model_filename = "stacking_classifier_model.joblib"
dump(stacking_clf, model_filename)
print(f"Model saved to {model_filename}")

Accuracy: 0.944944944944945
              precision    recall  f1-score   support

       blues       0.96      0.94      0.95       208
   classical       0.96      0.96      0.96       203
     country       0.90      0.90      0.90       186
       disco       0.96      0.94      0.95       199
      hiphop       0.96      0.95      0.96       218
        jazz       0.90      0.95      0.93       192
       metal       0.98      0.99      0.98       204
         pop       0.94      0.97      0.95       180
      reggae       0.96      0.94      0.95       211
        rock       0.93      0.91      0.92       197

    accuracy                           0.94      1998
   macro avg       0.94      0.94      0.94      1998
weighted avg       0.95      0.94      0.94      1998



In [3]:
# Load the model from disk
model_filename = 'stacking_classifier_model.joblib'
loaded_model = load(model_filename)
print("Model loaded")

# Use the loaded model to make predictions
y_pred_loaded = loaded_model.predict(X_test_scaled)
print(f"Accuracy of loaded model: {accuracy_score(y_test, y_pred_loaded)}")


XGBoostError: [16:41:14] C:\Users\dev-admin\croot2\xgboost-split_1675461376218\work\src\learner.cc:1220: Check failed: fp.Read(&header[0], header.size()) == serialisation_header_.size() (0 vs. 14) : 

In [1]:
# Correct the path to the new dataset
import pandas as pd

path = "data/features_3_sec.csv"  # Adjusted to use forward slashes

# Load the new dataset
df = pd.read_csv(path)